In [102]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from unidecode import unidecode
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import fasttext
import numpy as np


In [82]:
#!pip install fasttext

In [119]:
data = fetch_20newsgroups(subset='all', shuffle=True, remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({'text': data.data, 'label': data.target})
df = df[df['text'].apply(lambda x: len(x.strip()) > 0)]

In [120]:
len(df['label'].value_counts())

20

In [121]:
#df['text']

In [122]:
pd.set_option('display.max_colwidth', None)
df['text'].head(15)


0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [123]:
def preprocess_text(df_column):
    # Remove HTML tags
    df_column = df_column.apply(BeautifulSoup, args=('html.parser',)).apply(lambda x: x.get_text())

    # Removing accented characters
    df_column = df_column.apply(unidecode)
    df_column = df_column.str.lower()
    df_column = df_column.apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
    # Stemming
    #stemmer = PorterStemmer()
    #df_column = df_column.apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    lemmatizer = WordNetLemmatizer()
    df_column = df_column.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

    # Removing stop-words
    stop_words = set(stopwords.words('english'))
    df_column = df_column.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
    

    return df_column

In [124]:
df['text'] = preprocess_text(df['text'])

/home/fouzankhan/anaconda3/lib/python3.11/site-packages/pandas/core/apply.py:1278: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return func(x, *self.args, **self.kwargs)


In [70]:
#new_df = df

In [125]:
df = df[['label','text']]

In [126]:
df.head(6)

,label,text
0,10,sure bashers pen fan pretty confused lack kind post recent pen massacre devil actually bit puzzled bit relieved however going put end nonpittsburghers relief bit praise pen man killing devil worse thought jagr showed much better regular season stats also lot fo fun watch playoff bowman let jagr lot fun next couple game since pen going beat pulp jersey anyway wa disappointed see islander lose final regular season game pen rule
1,3,brother market highperformance video card support vesa local bus mb ram doe anyone suggestionsideas diamond stealth pro local bus orchid farenheit ati graphic ultra pro highperformance vlb card please post email thank matt
2,17,finally said dream mediterranean wa new area greater year like holocaust number ist july usa sweden april still cold changed calendar nothing mentioned true let say true shall azeri woman child going pay price raped killed tortured armenian hearded something called geneva convention facist ohhh forgot armenian fight nobody ha forgot killing rapings torture kurd turk upon time ohhhh swedish redcross worker lie ever say regional killer dont like person shoot thats policyl confused search turkish plane dont know talking turkey government ha announced giving weapon azerbadjan since armenia started attack azerbadjan self karabag province search plane weapon since content announced weapon one thats confused thats right give weapon azeri since armenian started fight azerbadjan shoot armenian bread butter arm personel russian army
3,3,think scsi card dma transfer disk scsi card dma transfer containing data scsi device attached want important feature scsi ability detach device free scsi bus device typically used multitasking start transfer several device device seeking data bus free command data transfer device ready transfer data aquire bus send data ide bus start transfer bus busy disk ha seeked data transfered typically second lock process wanting bus irrespective transfer time
4,4,old jasmine drive cannot use new system understanding upsate driver modern one order gain compatability system doe anyone know inexpensive program seen formatters buit idea work another ancient device one tape drive back utility freeze system try use drive jasmine direct tape bought used w tape techmar mechanism essentially question anyone know inexpensive beckup utility use system
5,12,back high school worked lab assistant bunch experimental psychologist bell lab visual perception memory experiment used vectortype display millisecond refresh rate common case th sec quite practical experimenter probably sure wa millisecond either steve


In [127]:
df.to_csv('cleaned_dataset.csv', index=False)

In [128]:
new_df = df

In [113]:
df['label'] = "__label__"+df['label'].astype(str)
df.head(2)

,label,text
0,__label__10,sure bashers pen fan pretty confused lack kind post recent pen massacre devil actually bit puzzled bit relieved however going put end nonpittsburghers relief bit praise pen man killing devil worse thought jagr showed much better regular season stats also lot fo fun watch playoff bowman let jagr lot fun next couple game since pen going beat pulp jersey anyway wa disappointed see islander lose final regular season game pen rule
1,__label__3,brother market highperformance video card support vesa local bus mb ram doe anyone suggestionsideas diamond stealth pro local bus orchid farenheit ati graphic ultra pro highperformance vlb card please post email thank matt


In [61]:
df['label_text'] = df['label']+ " "+ df['text']

In [14]:
train, test= train_test_split(df, test_size=0.33, random_state=42)

train.shape

(12281, 3)

In [15]:
test.shape

(6050, 3)

In [16]:
train.to_csv("newsgroup20.train",columns = ["label_text"], index=False, header=False)
train.to_csv("newsgroup20.test",columns = ["label_text"],index=False, header=False )

In [17]:
model = fasttext.train_supervised(input="newsgroup20.train")
model.test("newsgroup20.test")

Read 1M words
Number of words:  82655
Number of labels: 20
Progress: 100.0% words/sec/thread: 1785680 lr:  0.000000 avg.loss:  2.780288 ETA:   0h 0m 0s


(12281, 0.216676166435958, 0.216676166435958)

In [25]:
#df.dropna(inplace=True) df drop element

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['label'],test_size = 0.33, random_state = 96)

In [19]:
def get_doc_vectors(docs):
    vectors = [model.get_sentence_vector(doc) for doc in docs]
    return vectors

In [20]:
train_vectors = get_doc_vectors(X_train)

In [21]:
test_vectors = get_doc_vectors(X_test)

In [22]:
logreg_model = LogisticRegression()
logreg_model.fit(train_vectors, y_train)

/home/fouzankhan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [23]:
logreg_predictions = logreg_model.predict(test_vectors)
logreg_accuracy = accuracy_score(y_test, logreg_predictions)
logreg_classification_report = classification_report(y_test, logreg_predictions)

print("Logistic Regression Model:")
print(f"Accuracy: {logreg_accuracy}")
print("Classification Report:")
print(logreg_classification_report)

Logistic Regression Model:
Accuracy: 0.3852892561983471
Classification Report:
              precision    recall  f1-score   support

  __label__0       0.14      0.01      0.02       265
  __label__1       0.31      0.25      0.27       302
 __label__10       0.56      0.74      0.64       337
 __label__11       0.33      0.58      0.42       310
 __label__12       0.25      0.31      0.28       299
 __label__13       0.24      0.43      0.31       286
 __label__14       0.40      0.08      0.14       330
 __label__15       0.30      0.77      0.43       310
 __label__16       0.17      0.09      0.12       298
 __label__17       0.24      0.35      0.28       297
 __label__18       0.00      0.00      0.00       217
 __label__19       0.00      0.00      0.00       193
  __label__2       0.55      0.44      0.49       323
  __label__3       0.48      0.50      0.49       348
  __label__4       0.45      0.26      0.33       317
  __label__5       0.53      0.64      0.58       326
  

/home/fouzankhan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fouzankhan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fouzankhan/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [129]:
max_sequence_length = 60 #assuming this vlaue

In [130]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(new_df['text'],new_df['label'],test_size = 0.33, random_state = 96)

In [131]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train1)
train_sequences = tokenizer.texts_to_sequences(X_train1)
test_sequences = tokenizer.texts_to_sequences(X_test1)

In [132]:
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')
vocab_size = len(tokenizer.word_index) + 1

In [133]:
y_train1

13595     4
9803      2
16587     4
9742     18
1314     19
         ..
14182    10
9049     15
2483     12
9912      0
14834     4
Name: label, Length: 12281, dtype: int64

In [174]:
#from sklearn.preprocessing import LabelEncoder
#y_train_encoded = label_encoder.fit_transform(y_train)
#y_test_encoded = label_encoder.fit_transform(y_test)

In [134]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=350, input_length=max_sequence_length))
lstm_model.add(LSTM(350, return_sequences=True))
#lstm_model.add(Dropout(0.5))
lstm_model.add(LSTM(300))
#lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(200, activation='relu'))
lstm_model.add(Dense(20, activation='softmax'))
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(train_padded, y_train1, epochs=8, validation_data=(test_padded, y_test1))


Epoch 1/8
384/384 [==============================] - 256s 659ms/step - loss: 2.8604 - accuracy: 0.0893 - val_loss: 2.6902 - val_accuracy: 0.1099
Epoch 2/8
384/384 [==============================] - 263s 685ms/step - loss: 2.6316 - accuracy: 0.1145 - val_loss: 2.5440 - val_accuracy: 0.1393
Epoch 3/8
384/384 [==============================] - 262s 684ms/step - loss: 2.5316 - accuracy: 0.1352 - val_loss: 2.5907 - val_accuracy: 0.1301
Epoch 4/8
384/384 [==============================] - 295s 767ms/step - loss: 2.2947 - accuracy: 0.1791 - val_loss: 2.3459 - val_accuracy: 0.1815
Epoch 5/8
384/384 [==============================] - 308s 802ms/step - loss: 1.8709 - accuracy: 0.2978 - val_loss: 2.1277 - val_accuracy: 0.2668
Epoch 6/8
384/384 [==============================] - 325s 846ms/step - loss: 1.4397 - accuracy: 0.4546 - val_loss: 1.9879 - val_accuracy: 0.3679
Epoch 7/8
384/384 [==============================] - 293s 764ms/step - loss: 1.0029 - accuracy: 0.6328 - val_loss: 1.8687 - val_ac

In [ ]:
#this is my largest model of lstm
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=350, input_length=max_sequence_length))
lstm_model.add(LSTM(350, return_sequences=True))
#lstm_model.add(Dropout(0.5))
lstm_model.add(LSTM(300))
#lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(200, activation='relu'))
lstm_model.add(Dense(20, activation='softmax'))
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(train_padded, y_train1, epochs=20, validation_data=(test_padded, y_test1))


Epoch 1/20
384/384 [==============================] - 295s 762ms/step - loss: 2.7751 - accuracy: 0.0946 - val_loss: 2.6254 - val_accuracy: 0.1066
Epoch 2/20
384/384 [==============================] - 274s 713ms/step - loss: 2.5695 - accuracy: 0.1252 - val_loss: 2.6128 - val_accuracy: 0.1117
Epoch 3/20
384/384 [==============================] - 291s 757ms/step - loss: 2.5462 - accuracy: 0.1338 - val_loss: 2.4357 - val_accuracy: 0.1655
Epoch 4/20
384/384 [==============================] - 268s 697ms/step - loss: 2.1009 - accuracy: 0.2318 - val_loss: 2.1547 - val_accuracy: 0.2423
Epoch 5/20
384/384 [==============================] - 252s 655ms/step - loss: 1.6694 - accuracy: 0.3544 - val_loss: 1.8191 - val_accuracy: 0.3514
Epoch 6/20
384/384 [==============================] - 271s 707ms/step - loss: 1.2058 - accuracy: 0.5346 - val_loss: 1.6722 - val_accuracy: 0.4481
Epoch 7/20
384/384 [==============================] - 275s 716ms/step - loss: 0.8115 - accuracy: 0.6889 - val_loss: 1.7441 -

In [136]:
y_pred = lstm_model.predict(test_padded)
y_pred_class = [np.argmax(element) for element in y_pred]
acc = accuracy_score(y_test1, y_pred_class)
print(f"Accuracy: {acc * 100:.2f}%")
print(f"Confusion Matrix: {confusion_matrix(y_test1,y_pred_class)}")
lstm_classification_report = classification_report(y_test1, y_pred_class, digits=4)
print(lstm_classification_report)
print(y_pred)

190/190 [==============================] - 18s 93ms/step
Accuracy: 48.30%
Confusion Matrix: [[ 64   3   0   0   0   0   1   4   1   1   2  20   2  10   4  27  19   1
   71  35]
 [  0 111  27  28  30   4   6   4   5   2   0  16  44   5  20   0   0   0
    0   0]
 [  0  31 109 100  22  16  14   1   2   0   1   7  14   3   2   0   1   0
    0   0]
 [  0  22  25 162  51   3  53   4   1   0   0   3  21   0   2   0   0   0
    1   0]
 [  0  25   2  46 172   0  28   2   2   0   0   2  23   2  11   0   1   0
    1   0]
 [  0  23  94  11   3 166   1   1   1   1   0   8   8   2   5   0   0   0
    2   0]
 [  0   5   2  18   8   1 227  15  12   7   0   0  12   8   6   0   0   0
    0   1]
 [  0   4   0   2   0   0  53 159  48   8   0   3  20  10  12   0   6   0
    0   0]
 [  0   3   0   0   0   0  12  33 175  28   1   7  10  29  13   0  14   0
    1   1]
 [  0   4   0   0   0   0   1   4  26 197  29   2   4  30   3   0  13   0
    4   1]
 [  0   1   0   0   0   0   0   2   5  60 228   2   1  20 

In [137]:
# Evaluate LSTM Model the largest lstm model evaluations
lstm_loss, lstm_accuracy = lstm_model.evaluate(test_padded, y_test1)

print("\nLSTM Model:")
print(f"Accuracy: {lstm_accuracy}")

190/190 [==============================] - 17s 90ms/step - loss: 2.0296 - accuracy: 0.4830

LSTM Model:
Accuracy: 0.48297521471977234


In [ ]:
####accuracy for the 20 epoch version of lstm
y_pred = lstm_model.predict(test_padded)
y_pred_class = [np.argmax(element) for element in y_pred]
acc = accuracy_score(y_test1, y_pred_class)
print(f"Accuracy: {acc * 100:.2f}%")
print(f"Confusion Matrix: {confusion_matrix(y_test1,y_pred_class)}")
lstm_classification_report = classification_report(y_test1, y_pred_class, digits=4)
print(lstm_classification_report)
print(y_pred)

# Evaluate LSTM Model the largest lstm model evaluations
lstm_loss, lstm_accuracy = lstm_model.evaluate(test_padded, y_test1)

print("\nLSTM Model:")
print(f"Accuracy: {lstm_accuracy}")